In [1]:
import pandas as pd 

data = pd.read_csv("../datasets/attribute_set/in_shop_attr/in_shop_data_all.csv", index_col=0)

In [2]:
data.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53590 entries, 0 to 53589
Columns: 350 entries, path to 361_cutout
dtypes: int64(347), object(3)
memory usage: 143.5+ MB


In [3]:
data.head() 

,path,id,group,0_lightweight,1_polyester,2_woven,3_knit,4_cotton,5_unlined,6_rayon,...,352_vest,353_accessorie,354_brunch,355_girly,356_open-knit,357_polka,358_gauze,359_grey,360_outhwestern,361_cutout
0,../datasets/big_ds/img-in_shop/img/img/WOMEN/D...,id_00000002,train,1,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,../datasets/big_ds/img-in_shop/img/img/WOMEN/D...,id_00000002,train,1,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,../datasets/big_ds/img-in_shop/img/img/WOMEN/D...,id_00000002,train,1,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,../datasets/big_ds/img-in_shop/img/img/WOMEN/D...,id_00000002,train,1,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,../datasets/big_ds/img-in_shop/img/img/WOMEN/S...,id_00000003,train,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [4]:
data.tail() 

,path,id,group,0_lightweight,1_polyester,2_woven,3_knit,4_cotton,5_unlined,6_rayon,...,352_vest,353_accessorie,354_brunch,355_girly,356_open-knit,357_polka,358_gauze,359_grey,360_outhwestern,361_cutout
53585,../datasets/big_ds/img-in_shop/img/img/WOMEN/J...,id_00007982,gallery,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
53586,../datasets/big_ds/img-in_shop/img/img/WOMEN/J...,id_00007982,query,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
53587,../datasets/big_ds/img-in_shop/img/img/WOMEN/J...,id_00007982,gallery,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
53588,../datasets/big_ds/img-in_shop/img/img/WOMEN/J...,id_00007982,query,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
53589,../datasets/big_ds/img-in_shop/img/img/WOMEN/J...,id_00007982,query,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [5]:
TRAIN_SIZE = data[data["group"] == "train"].shape[0]
TRAIN_SIZE

26338

In [6]:
train_data_first = data[data["group"] == "train"].iloc[:14000, :]
train_data_second = data[data["group"] == "train"].iloc[14000:, :].sample(frac=1).reset_index(drop=True)
train_data_first.shape, train_data_second.shape 

((14000, 350), (12338, 350))

In [7]:
data[data["group"] == "gallery"].shape[0] 

12811

In [8]:
val_data_first = data[data["group"] == "gallery"].iloc[:2000,:] 
val_data_second = data[data["group"] == "gallery"].iloc[2000:4000, :] 
val_data_first.shape, val_data_second.shape 

((2000, 350), (2000, 350))

In [9]:
data[data["group"] == "query"].shape

(14441, 350)

In [10]:
data_first = pd.concat([train_data_first, val_data_first]) 
data_second = pd.concat([train_data_second, val_data_second]) 
data_first.shape, data_second.shape 

((16000, 350), (14338, 350))

In [11]:
data = data.drop_duplicates()

In [12]:
data.shape 

(52712, 350)

# Train 


In [42]:
data_second = data_second.drop_duplicates()
data = data_second

In [43]:
fnames = data.path.to_numpy()  
fnames[:5], len(fnames)

(array(['../datasets/big_ds/img-in_shop/img/img/WOMEN/Blouses_Shirts/id_00004472/02_3_back.jpg',
        '../datasets/big_ds/img-in_shop/img/img/WOMEN/Tees_Tanks/id_00007277/01_3_back.jpg',
        '../datasets/big_ds/img-in_shop/img/img/WOMEN/Shorts/id_00004829/05_4_full.jpg',
        '../datasets/big_ds/img-in_shop/img/img/WOMEN/Blouses_Shirts/id_00005469/01_4_full.jpg',
        '../datasets/big_ds/img-in_shop/img/img/WOMEN/Dresses/id_00007154/02_7_additional.jpg'],
       dtype=object),
 14179)

In [44]:
import tensorflow as tf 

ds_size = data.shape[0] 
number_of_selected_samples = 500 

#filelist_ds = tf.data.Dataset.from_tensor_slices(fnames[:number_of_selected_samples]) 
filelist_ds = tf.data.Dataset.from_tensor_slices(fnames) 


filelist_ds.cardinality().numpy() 

14179

## Custom tf Helpers 

In [45]:
def get_label(file_path):
    """
        file_path: the file path for the image that you want to select
    """
    labels = data.loc[data.path == file_path].to_numpy().squeeze()[3:].astype("int64")
    return tf.convert_to_tensor(labels) 

In [46]:
get_label(fnames[0])

<tf.Tensor: shape=(347,), dtype=int64, numpy=
array([0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
     

In [47]:
# resize and scale the images so that we can save time in training  
IMG_WIDTH, IMG_HEIGHT = 224, 224 
def decode_img(img):
    """
        img: img is the image 
    """ 
    #color images 
    img = tf.image.decode_jpeg(img, channels=3) 
    # img = tf.image.convert_image_dtype(img, tf.float32)  #convert unit8 tensor to floats in the [0,1] range
    img = tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT]) 
    # img = img / tf.constant(256, dtype=tf.float32)
    return img

In [48]:
def combine_images_labels(file_path: tf.Tensor): 
    label = get_label(file_path) 
    img = tf.io.read_file(file_path) 
    img = decode_img(img) 
    return img, label 

In [49]:
# combine_images_labels(fnames[0])

## Train/Test Split 

In [50]:
data.shape 

(14179, 350)

In [51]:
split_size = 12338
data_size = data.shape[0]
ds_train = filelist_ds.take(split_size) 
ds_test = filelist_ds.skip(split_size)

In [52]:
BATCH_SIZE = 32 

## Pre-process All the Images 

In [53]:
ds_train = ds_train.map(lambda x: 
                        tf.py_function(func=combine_images_labels, 
                                       inp=[x], # input of the function 
                                       Tout=(tf.float32,tf.int64)),  # return type 
                        num_parallel_calls=tf.data.AUTOTUNE, # parallelizing data extraction 
                        deterministic=False 
                        )

In [54]:
ds_test= ds_test.map(lambda x: tf.py_function(func=combine_images_labels,
          inp=[x], Tout=(tf.float32,tf.int64)),
          num_parallel_calls=tf.data.AUTOTUNE,
          deterministic=False)

### Prepare Data Pipeline 

- **batch**(): Combines consecutive elements of this dataset into batches.
- **cache**(): Caches the elements in this dataset. he first time the dataset is iterated over, its elements will be cached either in the specified file or in memory.Subsequent iterations will use the cached data.
- **prefetch**(): Creates a Dataset that prefetches elements from this dataset. Most dataset input pipelines should end with a call to *prefetch*. This allows later elements to be prepared while the current element is being processed. This often improves latency and throughput, at the cost of using additional memory to store prefetched elements.
 

In [55]:
ds_train_batched = ds_train.batch(BATCH_SIZE).cache().prefetch(tf.data.experimental.AUTOTUNE) 
ds_test_batched = ds_test.batch(BATCH_SIZE).cache().prefetch(tf.data.experimental.AUTOTUNE) 

In [56]:
ds_train_batched.cardinality().numpy() 

386

In [57]:
1318 * 32 

42176

In [58]:
nr_of_classes = len(data.columns) - 3  
nr_of_classes

347

# Model 0: simple CNN Model

In [27]:
from tensorflow import keras 

inputs = keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)) 
x = keras.layers.Conv2D(filters=32,
                        kernel_size=5,
                        activation='relu')(inputs) 
x = keras.layers.Conv2D(filters=32,
                        kernel_size=5,
                        activation='relu')(x) 
x = keras.layers.GlobalMaxPool2D(name="max_pooling")(x) 
outputs = keras.layers.Dense(nr_of_classes, name="output_of_cnn")(x)

deneme_model = tf.keras.models.Model(inputs, outputs)

In [28]:
deneme_model.compile(optimizer=keras.optimizers.Adam(), 
                    loss=keras.losses.BinaryCrossentropy(),
                    metrics=[keras.metrics.BinaryAccuracy()])

In [29]:
history_model_den = deneme_model.fit(ds_train_batched, 
        validation_data =ds_test_batched,
        epochs=10)

Epoch 1/10
1318/1318 [==============================] - ETA: 0s - loss: 5.4009 - binary_accuracy: 0.6461  

## Model 1:Fine Grained VGG16 

In [59]:
from tensorflow import keras 
base_model = keras.applications.VGG16(
    weights="imagenet", # load weights pre-trained on ImageNet. 
    input_shape=(IMG_WIDTH, IMG_HEIGHT, 3), # VGG16 expects min 32 x 32 
    include_top = False # do not include output layer of the image net vgg 
)
base_model.trainable = False 

In [60]:
inputs = keras.Input(shape=(IMG_WIDTH,IMG_HEIGHT,3))
x = tf.cast(inputs, tf.float32)
x = tf.keras.applications.vgg16.preprocess_input(x)   
x = base_model(x) 
x = keras.layers.GlobalAveragePooling2D()(x) 

initializer = tf.keras.initializers.GlorotUniform(seed=42) 
activation = tf.keras.activations.sigmoid  

outputs = keras.layers.Dense(nr_of_classes,
                             kernel_initializer=initializer, 
                             activation=activation)(x) 

model_1 = keras.Model(inputs, outputs) 

In [61]:
model_1.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(), # default from_logits=False
              metrics=[keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall')])

In [62]:
previous_checkpoint = "checkpoints/in-shop_model_1/checkpoint_VGG_Model_1.ckpt"

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5) 
checkpoint_path = "checkpoints/in-shop_model_1_STEP2/checkpoint_VGG_Model_1.ckpt" 
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                         save_weights_only=True,
                                                         save_best_only=False,
                                                         verbose=1)

In [63]:
model_1.load_weights(previous_checkpoint)

In [64]:
history_model_1 = model_1.fit(ds_train_batched, 
                            epochs=10,
                            validation_data =ds_test_batched,
                            callbacks=[early_stopping, checkpoint_callback]
                            )

Epoch 1/10
386/386 [==============================] - ETA: 0s - loss: 0.1660 - tp: 62641.0000 - fp: 41268.0000 - tn: 4003056.0000 - fn: 174321.0000 - accuracy: 0.9496 - precision: 0.6028 - recall: 0.2644  
Epoch 1: saving model to checkpoints/in-shop_model_1_STEP2\checkpoint_VGG_Model_1.ckpt
386/386 [==============================] - 52140s 135s/step - loss: 0.1660 - tp: 62641.0000 - fp: 41268.0000 - tn: 4003056.0000 - fn: 174321.0000 - accuracy: 0.9496 - precision: 0.6028 - recall: 0.2644 - val_loss: 0.1621 - val_tp: 9222.0000 - val_fp: 5312.0000 - val_tn: 598846.0000 - val_fn: 25447.0000 - val_accuracy: 0.9519 - val_precision: 0.6345 - val_recall: 0.2660
Epoch 2/10
386/386 [==============================] - ETA: 0s - loss: 0.1509 - tp: 65404.0000 - fp: 34301.0000 - tn: 4010023.0000 - fn: 171558.0000 - accuracy: 0.9519 - precision: 0.6560 - recall: 0.2760
Epoch 2: saving model to checkpoints/in-shop_model_1_STEP2\checkpoint_VGG_Model_1.ckpt
386/386 [==============================] - 7

In [65]:
model_1.save("../trained_models/IN_SHOP_Models/model_1_vgg") 

INFO:tensorflow:Assets written to: ../trained_models/IN_SHOP_Models/model_1_vgg\assets


INFO:tensorflow:Assets written to: ../trained_models/IN_SHOP_Models/model_1_vgg\assets


In [66]:
pd.DataFrame(history_model_1.history).to_csv("../trained_models/IN_SHOP_Models/model_1_vgg_RESULT.csv")

## Model 2: Efficient Net 

In [67]:
efficient_net = tf.keras.applications.EfficientNetB0(include_top=False)
efficient_net.trainable = False

In [68]:
from tensorflow import keras 
inputs = keras.Input(shape=(IMG_WIDTH,IMG_HEIGHT,3)) 
x = efficient_net(inputs) 
x = keras.layers.GlobalAveragePooling2D()(x) 

initializer = tf.keras.initializers.GlorotUniform(seed=42) 
activation = tf.keras.activations.sigmoid  

outputs = keras.layers.Dense(nr_of_classes,
                             kernel_initializer=initializer, 
                             activation=activation)(x) 

model_2 = keras.Model(inputs, outputs) 

In [69]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall')
]

model_2.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(), # default from_logits=False
              metrics=METRICS)

In [70]:
model_2.load_weights("checkpoints/in-shop_ds_model2/checkpoint_Efficient_Base.ckpt")

In [71]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5) 
checkpoint_path = "checkpoints/in-shop_ds_model2_STEP_2/checkpoint_Efficient_Base.ckpt" 
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                         save_weights_only=True,
                                                         save_best_only=False,
                                                         verbose=1)

In [72]:
history_model_2 = model_2.fit(ds_train_batched, 
        validation_data =ds_test_batched,
        epochs=10,
        callbacks=[early_stopping, checkpoint_callback])

Epoch 1/10
386/386 [==============================] - ETA: 0s - loss: 0.1496 - tp: 63346.0000 - fp: 28537.0000 - tn: 4015787.0000 - fn: 173616.0000 - accuracy: 0.9528 - precision: 0.6894 - recall: 0.2673
Epoch 1: saving model to checkpoints/in-shop_ds_model2_STEP_2\checkpoint_Efficient_Base.ckpt
386/386 [==============================] - 291s 721ms/step - loss: 0.1496 - tp: 63346.0000 - fp: 28537.0000 - tn: 4015787.0000 - fn: 173616.0000 - accuracy: 0.9528 - precision: 0.6894 - recall: 0.2673 - val_loss: 0.1440 - val_tp: 9814.0000 - val_fp: 3646.0000 - val_tn: 600512.0000 - val_fn: 24855.0000 - val_accuracy: 0.9554 - val_precision: 0.7291 - val_recall: 0.2831
Epoch 2/10
386/386 [==============================] - ETA: 0s - loss: 0.1428 - tp: 65136.0000 - fp: 26515.0000 - tn: 4017809.0000 - fn: 171826.0000 - accuracy: 0.9537 - precision: 0.7107 - recall: 0.2749
Epoch 2: saving model to checkpoints/in-shop_ds_model2_STEP_2\checkpoint_Efficient_Base.ckpt
386/386 [==========================

In [78]:
model_2.save("../trained_models/IN_SHOP_Models/model_2_efficient_net") 

INFO:tensorflow:Assets written to: ../trained_models/IN_SHOP_Models/model_2_efficient_net\assets


INFO:tensorflow:Assets written to: ../trained_models/IN_SHOP_Models/model_2_efficient_net\assets


In [77]:
pd.DataFrame(history_model_2.history).to_csv("../trained_models/IN_SHOP_Models/model_2_efficient_net.csv")

### Model 2 Fine Tune

In [79]:
from tensorflow.keras.models import clone_model 

model_2_fine = clone_model(model_2)
model_2_fine.set_weights(model_2.get_weights())

In [80]:
for layer in model_2_fine.layers: 
    print(layer.trainable,"name:" ,layer.name)

True name: input_6
True name: efficientnetb0
True name: global_average_pooling2d_2
True name: dense_2


In [81]:
ef_net_for_fine = model_2_fine.layers[1]
model_2_fine.layers[1].name 

'efficientnetb0'

In [82]:
ef_net_for_fine.trainable = True  

for layer in ef_net_for_fine.layers[:-10]:
    layer.trainable = False 

In [83]:
model_2_fine.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(), 
              metrics=[keras.metrics.BinaryAccuracy()])

In [84]:
for layer in ef_net_for_fine.layers[-10:]: 
    print(layer.trainable, layer.name)

True block7a_se_squeeze
True block7a_se_reshape
True block7a_se_reduce
True block7a_se_expand
True block7a_se_excite
True block7a_project_conv
True block7a_project_bn
True top_conv
True top_bn
True top_activation


In [85]:
model_2_fine.load_weights("checkpoints/in-shop_ds_model2_fine_tune/checkpoint_Efficient_Base.ckpt" )

In [86]:
checkpoint_path = "checkpoints/in-shop_ds_model2_fine_tune_STEP2/checkpoint_Efficient_Base.ckpt" 
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                         save_weights_only=True,
                                                         save_best_only=False,
                                                         verbose=1)

In [87]:
history_model_2_fine = model_2.fit(ds_train_batched, 
        validation_data =ds_test_batched,
        epochs=20, 
        initial_epoch=history_model_2.epoch[-1], # start from previous last epoch 
        callbacks=[early_stopping, checkpoint_callback])

Epoch 10/20
 56/386 [===>..........................] - ETA: 3:36 - loss: 0.1231 - tp: 11296.0000 - fp: 3735.0000 - tn: 583397.0000 - fn: 23396.0000 - accuracy: 0.9564 - precision: 0.7515 - recall: 0.3256

In [ ]:
int(len(ef_net_for_fine.layers) / 3)

optimizers = [
    keras.optimizers.Adam(), # for layer [0] and last layer 
    tf.keras.optimizers.Adam(learning_rate=1e-6), # ef net [0 to 80] 
    tf.keras.optimizers.Adam(learning_rate=1e-4), # ef net [80 to 160] 
    tf.keras.optimizers.Adam(learning_rate=1e-2), # # ef net [160 to :ef net last] 
]

In [ ]:
optimizers_and_layers = [
    (optimizers[0], model_2_fine.layers[0]), 
    (optimizers[1], ef_net_for_fine.layers[0:80]), 
    (optimizers[2], ef_net_for_fine.layers[80:160]),
    (optimizers[3], ef_net_for_fine.layers[160:]),
    (optimizers[0], model_2_fine.layers[-1]),   
]

In [ ]:
#!pip install tensorflow-addons

In [ ]:
import tensorflow_addons as tfa
optimizer = tfa.optimizers.MultiOptimizer(optimizers_and_layers)